In [ ]:
#default_exp lightning.callbacks

In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# export
import sys
import time
import datetime
import logging
from collections import namedtuple
from tqdm.auto import tqdm

import wandb

import torch
import pytorch_lightning as pl
from pytorch_lightning import _logger as log

from src.lightning.core import *
from src.core import *

<IPython.core.display.Javascript object>

In [ ]:
# export
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

In [ ]:
# export
class WandbImageClassificationCallback(pl.Callback):
    """ Custom callback to add some extra functionalites to the wandb logger """

    def __init__(
        self,
        num_batches: int = 16,
        log_train_batch: bool = False,
        log_preds: bool = False,
        log_conf_mat: bool = True,
    ):

        # class names for the confusion matrix
        self.class_names = list(conf_mat_idx2lbl.values())

        # counter to log training batch images
        self.num_bs = num_batches
        self.curr_epoch = 0

        self.log_train_batch = log_train_batch
        self.log_preds = log_preds
        self.log_conf_mat = log_conf_mat

        self.val_imgs, self.val_labels = None, None

    def on_train_start(self, trainer, pl_module, *args, **kwargs):
        try:
            # log model to the wandb experiment
            wandb.watch(models=pl_module.model, criterion=pl_module.loss_func)
        except:
            logger.info("Skipping wandb.watch --->")

    def on_train_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_train_batch:
            if pl_module.one_batch is None:
                logger.info(
                    f"{self.config_defaults['mixmethod']} samples not available . Skipping --->"
                )
                pass

            else:
                one_batch = pl_module.one_batch[: self.num_bs]
                train_ims = one_batch.data.to("cpu")
                trainer.logger.experiment.log(
                    {"train_batch": [wandb.Image(x) for x in train_ims]}, commit=False
                )

    def on_validation_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_preds:
            if self.val_imgs is None and self.val_labels is None:
                self.val_imgs, self.val_labels = next(iter(pl_module.val_dataloader()))
                self.val_imgs, self.val_labels = (
                    self.val_imgs[: self.num_bs],
                    self.val_labels[: self.num_bs],
                )
                self.val_imgs = self.val_imgs.to(device=pl_module.device)

            logits = pl_module(self.val_imgs)
            preds = torch.argmax(logits, 1)
            preds = preds.data.cpu()

            ims = [
                wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                for x, pred, y in zip(self.val_imgs, preds, self.val_labels)
            ]
            log_dict = {"predictions": ims}
            wandb.log(ims, commit=False)

    def on_epoch_start(self, trainer, pl_module, *args, **kwargs):
        pl_module.val_labels_list = []
        pl_module.val_preds_list = []

    def on_epoch_end(self, trainer, pl_module, *args, **kwargs):
        if self.log_conf_mat:
            val_preds = torch.tensor(pl_module.val_preds_list).data.cpu().numpy()
            val_labels = torch.tensor(pl_module.val_labels_list).data.cpu().numpy()
            log_dict = {
                "conf_mat": wandb.plot.confusion_matrix(
                    val_preds, val_labels, self.class_names
                )
            }
            wandb.log(log_dict, commit=False)

<IPython.core.display.Javascript object>

In [ ]:
# export
class LitProgressBar(pl.callbacks.ProgressBar):
    "Custom Progressbar callback for Lightning Training"

    def init_sanity_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for the validation sanity run. """
        bar = tqdm(
            desc="Validation sanity check",
            position=(2 * self.process_position),
            disable=self.is_disabled,
            dynamic_ncols=True,
        )

        return bar

    def init_train_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for training. """
        bar = tqdm(
            desc="Training",
            initial=self.train_batch_idx,
            position=(2 * self.process_position),
            disable=self.is_disabled,
            dynamic_ncols=True,
        )

        return bar

    def init_validation_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for validation. """
        bar = tqdm(
            desc="Validating",
            position=(2 * self.process_position + 1),
            disable=True,
            dynamic_ncols=False,
        )

        return bar

    def init_test_tqdm(self) -> tqdm:
        """ Override this to customize the tqdm bar for testing. """
        bar = tqdm(
            desc="Testing",
            position=(2 * self.process_position),
            disable=self.is_disabled,
            dynamic_ncols=True,
        )

        return bar

<IPython.core.display.Javascript object>

In [ ]:
# export
class PrintLogsCallback(pl.Callback):
    "Logs Training logs to console after every epoch"

    def __init__(self):
        self.TestResult = namedtuple("TestResult", ["test_loss", "test_acc"])
        self.TrainResult = namedtuple(
            "TrainResult", ["loss", "acc", "valid_loss", "valid_acc"]
        )
        self.logger = logger

    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        train_loss = metrics["train/loss_epoch"]
        train_acc = metrics["train/acc_epoch"]
        valid_loss = metrics["valid/loss"]
        valid_acc = metrics["valid/acc"]
        trn_res = self.TrainResult(
            round(train_loss.data.cpu().numpy().item(), 2),
            round(train_acc.data.cpu().numpy().item(), 2),
            round(valid_loss.data.cpu().numpy().item(), 2),
            round(valid_acc.data.cpu().numpy().item(), 2),
        )

        curr_epoch = int(trainer.current_epoch)
        self.logger.info(f"[{curr_epoch}]: (100.00% done), {trn_res}")

    def on_test_epoch_end(self, trainer, pl_module, *args, **kwargs):
        metrics = trainer.callback_metrics
        test_loss = metrics["test/loss"]
        test_acc = metrics["test/acc"]
        self.logger.info(
            f"{self.TestResult(round(test_loss.data.cpu().numpy().item(), 2), round(test_acc.data.cpu().numpy().item(), 2))}"
        )

<IPython.core.display.Javascript object>

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
from torch import nn
from src.networks import *
from omegaconf import OmegaConf

<IPython.core.display.Javascript object>

In [ ]:
train_augs = A.Compose(
    [
        A.RandomResizedCrop(224, 224, p=1.0),
        A.RandomBrightness(limit=0.1),
        A.HueSaturationValue(20, 20, 20),
        A.HorizontalFlip(),
        A.Normalize(p=1.0),
        ToTensorV2(p=1.0),
    ]
)

valid_augs = A.Compose(
    [A.Resize(224, 224, p=1.0), A.Normalize(p=1.0), ToTensorV2(p=1.0)]
)

csv = "../../leaf-disease-classification-kaggle/data/stratified-data-5folds.csv"
ims = "../../Datasets/cassava/train_images/"
dm = CassavaLightningDataModule(
    csv,
    ims,
    curr_fold=0,
    train_augs=train_augs,
    valid_augs=valid_augs,
    bs=8,
    num_workers=0,
)


model_hparams = dict(
    mixmethod=None,
    loss=dict(_target_="src.losses.LabelSmoothingCrossEntropy", eps=0.1),
    learning_rate=1e-03,
    lr_mult=100,
    optimizer=dict(_target_="torch.optim.Adam"),
    scheduler=dict(
        function=dict(_target_="src.opts.FlatCos", num_epochs=10, pct_start=0.7),
        metric_to_track=None,
        scheduler_interval="step",
    ),
)

cfg = OmegaConf.create(model_hparams)


encoder = timm.create_model("resnet18", pretrained=False)
model = TransferLearningModel(encoder, cut=-2, c=5, act=nn.ReLU(inplace=True))
model = LightningCassava(model=model, conf=cfg)

Loss Function : LabelSmoothingCrossEntropy()


<IPython.core.display.Javascript object>

In [ ]:
trainer = pl.Trainer(
    callbacks=[LitProgressBar(), PrintLogsCallback()],
    num_sanity_val_steps=0,
    max_epochs=2,
    limit_train_batches=1,
    limit_val_batches=1,
    limit_test_batches=1,
    weights_summary=None,
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


<IPython.core.display.Javascript object>

In [ ]:
trainer.fit(model, datamodule=dm)

Data(fold=0, batch_size=8, im_path='../../Datasets/cassava/train_images/')
Optimizer(optimizer='Adam', scheduler='FlatCos', lrs=Lrs(encoder_lr=1e-05, fc_lr=0.001), wd=None)


[0]: (100.00% done), TrainResult(loss=1.73, acc=0.25, valid_loss=1.61, valid_acc=0.0)
[1]: (100.00% done), TrainResult(loss=1.57, acc=0.38, valid_loss=1.59, valid_acc=0.75)


1

<IPython.core.display.Javascript object>

In [ ]:
_ = trainer.test(model, datamodule=dm, verbose=False)

TestResult(test_loss=1.59, test_acc=0.75)


<IPython.core.display.Javascript object>

In [ ]:
# hide
from nbdev.export import *

notebook2script()

Converted 00_core.ipynb.
Converted 01_mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03_layers.ipynb.
Converted 03a_networks.ipynb.
Converted 04_optimizers_schedules.ipynb.
Converted 05_lightning.core.ipynb.
Converted 05a_lightning.callbacks.ipynb.
Converted 06_fastai.core.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>